## Combined Benchmark with Anomaly Data

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as sp_stats
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import LZH_Utilities as utl

### Constants

In [ ]:
DATA_FILE_PATH = "full_rank_dataset_anormaly"
OUTPUT_FOLDER_PATH = "Output/combined_benchmark_anomaly/"
OUTPUT_FILE_PREFIX = "y_hat_"

### Functions

In [ ]:
def save_result(file_name, data):
    """
    data as {
        "y_hat":     nparray,
        "y_hat_ERA5":nparray,
        "GRU":       nparray,
        "GRU_FB":    nparray,
        "FC":        nparray,
        "FC_FB":     nparray,
        "Linear":    nparray,
        "Linear_FB": nparray,
        "ERA5":      nparray,
    }
    """
    with open(file_name, 'wb') as f:
        np.save(f, data["y_hat"])
        np.save(f, data["y_hat_ERA5"])
        np.save(f, data["GRU"])
        np.save(f, data["GRU_FB"])
        np.save(f, data["FC"])
        np.save(f, data["FC_FB"])
        np.save(f, data["Linear"])
        np.save(f, data["Linear_FB"])
        np.save(f, data["ERA5"])
        
    return file_name
        
def load_result(file_name):
    """
    return data as {
        "y_hat":     nparray,
        "y_hat_ERA5":nparray,
        "GRU":       nparray,
        "GRU_FB":    nparray,
        "FC":        nparray,
        "FC_FB":     nparray,
        "Linear":    nparray,
        "Linear_FB": nparray,
        "ERA5":      nparray,
    }
    """
    data = {}
    with open(file_name, 'rb') as f:
        data["y_hat"] = np.load(f)
        data["y_hat_ERA5"] = np.load(f)
        data["GRU"] = np.load(f)
        data["GRU_FB"] = np.load(f)
        data["FC"] = np.load(f)
        data["FC_FB"] = np.load(f)
        data["Linear"] = np.load(f)
        data["Linear_FB"] = np.load(f)
        data["ERA5"] = np.load(f)
    return data

In [ ]:
def transpose_stack(arr):
    return np.array([arr]).T

def tsr(arr):
    return torch.tensor(arr)

def plot(x, y, x_label="", y_label="", legend="", title=""):
    # plt.figure(figsize=[8, 6], dpi=300)
    plt.figure(figsize=[8, 6])
    
    if (type(legend) is list):
        for yy in y:
            plt.plot(x, yy)
        plt.legend(legend)
    else: 
        plt.plot(x, y)
        
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    plt.show()

In [ ]:
def calc_r_value(x, y):
    return sp_stats.linregress(x, y).rvalue

In [ ]:
def plot_diff_percentage(x, y, x_label="", y_label="", title=""):
    return # do not output

    # plt.figure(figsize=[8, 6], dpi=300)
    plt.figure(figsize=[8, 6])
    
    x_fit = np.linspace(0, 100, 1000)
    y_fit1 = np.polyval(np.polyfit(x, y, 1), x_fit)
    
    plt.scatter(x, y, s=0.5, c='k')
    plt.plot(x_fit, y_fit1, "r")
        
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend(["linear regression", "data"])
    plt.ylim([0, 100])
    plt.xlim([0, 100])
    plt.title(title)
    plt.show()
    
    result = sp_stats.linregress(x, y)
    print("     slope: {0}".format(result.slope))
    print(" intercept: {0}".format(result.intercept))
    print("corr coeff: {0}".format(result.rvalue))
    print("  variance: {0}".format(result.rvalue ** 2))

In [ ]:
def plot_diff_percentage_bined_average(x, y, x_label="", y_label="", title="", bin_count=10):
    return # do not output

    # plt.figure(figsize=[8, 6], dpi=300)
    plt.figure(figsize=[8, 6])
    
    # Overall
    x_fit = np.linspace(0, 100, 10)
    y_fit1 = np.polyval(np.polyfit(x, y, 1), x_fit)
    
    plt.scatter(x, y, s=0.5, alpha=0.3, c='k', label="data")
    plt.plot(x_fit, y_fit1, "r", label="linear regression")
        
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    
    result = sp_stats.linregress(x, y)
    print("Overall:")
    print("     slope: {0}".format(result.slope))
    print(" intercept: {0}".format(result.intercept))
    print("corr coeff: {0}".format(result.rvalue))
    print("  variance: {0}".format(result.rvalue ** 2))
    print()
    
    # Binned Average
    is_label_printed = False
    
    bin_arr = np.linspace(0, 100, bin_count+1)
    for (low, high) in zip(bin_arr[:-1], bin_arr[1:]):
        
        bin_filter = np.logical_and(low <= x, x <= high)
        x_bin = x[bin_filter]
        y_bin = y[bin_filter]
        
        x_fit = np.linspace(low, high, 10)
        y_fit1 = np.polyval(np.polyfit(x_bin, y_bin, 1), x_fit)
        
        if (not is_label_printed):
            plt.plot(x_fit, y_fit1, "b", label="binned linear reg.")
            is_label_printed = True
        else:
            plt.plot(x_fit, y_fit1, "b")
            
        result = sp_stats.linregress(x_bin, y_bin)
        print("[{4:>3}, {5:>3}):   b:{0:.4f};  m:{1:.4f}; r:{2:.4f}; var:{3:.4f}".format(result.slope, result.intercept, result.rvalue, result.rvalue ** 2, int(low), int(high)))
        

            
    plt.legend()
    plt.ylim([0, 100])
    plt.xlim([0, 100])
    plt.title(title)
    plt.show()

In [ ]:
def get_training_and_testing_set():
    """
    return format: (idx_training_set, idx_test_set)
    """
    df = utl.read_time_series_data(DATA_FILE_PATH)
    idx_test_set = np.random.choice(np.arange(df[0].shape[0]), [int(0.1 * df[0].shape[0])], False)
    idx_training_set = np.delete(np.arange(df[0].shape[0]), idx_test_set)
    
    return (idx_training_set, idx_test_set)

In [ ]:
def get_data(idx_test_set, idx_training_set):
    """
    return format: ((X_train, y_hat_train), (X_test, y_hat_test))
    """
    df = utl.read_time_series_data(DATA_FILE_PATH)

    df[0]["TCC_FB"] = 0
    for idx in np.arange(8):
        df[idx+1]["TCC_FB"] = df[idx]["TCC"]
    
    # assigning new order
    new_col = ['LTS', 'SST', 'Subsidence', 'Night_Day', 'RH', 'q', 'wsp',
               'LTS_A', 'SST_A', 'Subsidence_A', 'RH_A', 'q_A', 'wsp_A', 'TCC']

    for idx in np.arange(len(df)):
        df[idx] = df[idx][new_col]
    
    time_arr = np.arange(9)

    X_full = [df[time].iloc[:, :-1].to_numpy() for time in time_arr]
    y_hat_full = [np.c_[df[time].iloc[:, -1].to_numpy()] for time in time_arr]

    X_train = np.array([X_full[time][idx_training_set] for time in time_arr])
    y_hat_train = np.array([y_hat_full[time][idx_training_set] for time in time_arr])

    X_test = np.array([X_full[time][idx_test_set] for time in time_arr])
    y_hat_test = np.array([y_hat_full[time][idx_test_set] for time in time_arr])
    return ((X_train, y_hat_train), (X_test, y_hat_test))

def get_data_TCC_feedback(idx_test_set, idx_training_set):
    """
    return format: ((X_train, y_hat_train), (X_test, y_hat_test))
    """
    df = utl.read_time_series_data(DATA_FILE_PATH)
    df[0]["TCC_FB"] = 0
    for idx in np.arange(8):
        df[idx+1]["TCC_FB"] = df[idx]["TCC"]
    
    # assigning new order
    new_col = ['LTS', 'SST', 'Subsidence', 'Night_Day', 'RH', 'q', 'wsp',
               'LTS_A', 'SST_A', 'Subsidence_A', 'RH_A', 'q_A', 'wsp_A', 'TCC_FB', 'TCC']

    for idx in np.arange(len(df)):
        df[idx] = df[idx][new_col]
    
    time_arr = np.arange(9)

    X_full = [df[time].iloc[:, :-1].to_numpy() for time in time_arr]
    y_hat_full = [np.c_[df[time].iloc[:, -1].to_numpy()] for time in time_arr]

    X_train = np.array([X_full[time][idx_training_set] for time in time_arr])
    y_hat_train = np.array([y_hat_full[time][idx_training_set] for time in time_arr])

    X_test = np.array([X_full[time][idx_test_set] for time in time_arr])
    y_hat_test = np.array([y_hat_full[time][idx_test_set] for time in time_arr])
    
    return ((X_train, y_hat_train), (X_test, y_hat_test))

In [ ]:
def flatten_time_axis(data):
    """
    flatten the time axis so it could be used in FC and linear model. 
    """
    return np.concatenate(data, axis=0)

***
### Model

#### GRU

In [ ]:
class MLTCC_Model_GRU_Normalized:
    def __init__(self):
        self.NMR = utl.Normalizer()
        
        self.input_size = 13
        self.output_size = 1
        self.batch_size = 10000
        self.step_size = 1e-3
        
        h_gru = 50
        h1 = 20
        h2 = 40
        h3 = 20
        
        self.gru = nn.GRU(self.input_size, h_gru, 2, batch_first=False)   # (seq, batch, feature)
        self.fc1 = nn.Linear(h_gru, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, self.output_size)
        
        variable_list = [
                self.fc1.weight, self.fc1.bias,
                self.fc2.weight, self.fc2.bias,
                self.fc3.weight, self.fc3.bias,
                self.fc4.weight, self.fc4.bias
            ]
        for all_weights in self.gru.all_weights:
            for weights in all_weights:
                variable_list.append(weights)
        
        
        self.optim = torch.optim.Adam(
            variable_list,
            lr=self.step_size
        )
        
    def foward(self, X):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
        y := [TCC],                  (sample_size, 1) like tensor
        """
#         seq_X_result = []
#         for seq_X in X:
#             seq_X_result.append(torch.relu(self.fc1(seq_X.float())))
#         seq_X_result = tsr(seq_X_result)
        # normalize
        newX = []
        for idx in np.arange(X.shape[0]):
            newX.append(self.NMR.normalize_input(X[idx]).detach().numpy())
        X = torch.tensor(np.array(newX))

        X = X.float()
    
        X_gru_outputs, hn = self.gru(X)
        X = torch.relu(X_gru_outputs[-1]) # use last output
        
        X = torch.relu(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = torch.relu(self.fc3(X))
        X = self.fc4(X)
        
        return self.NMR.denormalize_output(X)
    
    def loss(self, y, y_hat):
        """
        y := [TCC],                  (sample_size, 1) like tensor
        y_hat := [TCC],              (sample_size, 1) like tensor
        """
        return F.mse_loss(y.float(), y_hat.float())
    
    def train(self, X, y_hat, max_epoch):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        sample_count = X.shape[1]
        
        epoch_num = []
        losses = []
        
        for i in np.arange(max_epoch):
            for idx_batch in np.arange(int(sample_count / self.batch_size)) * self.batch_size:
                X_batch = torch.tensor(X[:, idx_batch:idx_batch + self.batch_size, :])
                y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch.t(), y_hat_batch.t())
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                
            idx_rand = np.random.randint(0, sample_count, [self.batch_size])
            X_batch = torch.tensor(X[:, idx_rand, :]).float()
            y_hat_batch = torch.tensor(y_hat[idx_rand, :]).float()
            y_batch = self.foward(X_batch)
            loss = self.loss(y_batch, y_hat_batch)
            
            epoch_num.append(i)
            losses.append(loss.item())
        
        return (epoch_num, losses)
    
    
    def train_time_series(self, X, y_hat, max_epoch, max_epoch_per_time):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")
        
        # setting normalizer
        # flatten X and y_hat
        X_flat = np.concatenate(X, axis=0)
        y_hat_flat = np.concatenate(y_hat, axis=0)
        
        self.NMR.set_mean_and_sd(torch.tensor(X_flat), torch.tensor(y_hat_flat))
        
        
        full_losses = np.array([])

        T_running = utl.get_runtime_marker()
        for i in np.arange(max_epoch):
            for time in np.arange(9):
                (x_axis, losses) = self.train(X[:time+1, :, :], y_hat[time, :, :], max_epoch_per_time)
                full_losses = np.concatenate((full_losses, losses))
            progress_bar.update(i + 1, "epoch {0}".format(i))
            
        return (np.arange(len(full_losses)) + 1, full_losses)

#### GRU with TCC Feedback

In [ ]:
class MLTCC_Model_GRU_Normalized_TCC:
    def __init__(self):
        self.NMR = utl.Normalizer()
        
        self.input_size = 14
        self.output_size = 1
        self.batch_size = 10000
        self.step_size = 1e-3
        
        h_gru = 50
        h1 = 20
        h2 = 40
        h3 = 20
        
        self.gru = nn.GRU(self.input_size, h_gru, 2, batch_first=False)   # (seq, batch, feature)
        self.fc1 = nn.Linear(h_gru, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.fc4 = nn.Linear(h3, self.output_size)
        
        variable_list = [
                self.fc1.weight, self.fc1.bias,
                self.fc2.weight, self.fc2.bias,
                self.fc3.weight, self.fc3.bias,
                self.fc4.weight, self.fc4.bias
            ]
        for all_weights in self.gru.all_weights:
            for weights in all_weights:
                variable_list.append(weights)
        
        
        self.optim = torch.optim.Adam(
            variable_list,
            lr=self.step_size
        )
        
    def foward(self, X):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
        y := [TCC],                  (sample_size, 1) like tensor
        """
#         seq_X_result = []
#         for seq_X in X:
#             seq_X_result.append(torch.relu(self.fc1(seq_X.float())))
#         seq_X_result = tsr(seq_X_result)
        # normalize
        newX = []
        for idx in np.arange(X.shape[0]):
            newX.append(self.NMR.normalize_input(X[idx]).detach().numpy())
        X = torch.tensor(np.array(newX))

        X = X.float()
    
        X_gru_outputs, hn = self.gru(X)
        X = torch.relu(X_gru_outputs[-1]) # use last output
        
        X = torch.relu(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = torch.relu(self.fc3(X))
        X = self.fc4(X)
        
        return self.NMR.denormalize_output(X)
    
    def loss(self, y, y_hat):
        """
        y := [TCC],                  (sample_size, 1) like tensor
        y_hat := [TCC],              (sample_size, 1) like tensor
        """
        return F.mse_loss(y.float(), y_hat.float())
    
    def train(self, X, y_hat, max_epoch):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        sample_count = X.shape[1]
        
        epoch_num = []
        losses = []
        
        for i in np.arange(max_epoch):
            for idx_batch in np.arange(int(sample_count / self.batch_size)) * self.batch_size:
                X_batch = torch.tensor(X[:, idx_batch:idx_batch + self.batch_size, :])
                y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch.t(), y_hat_batch.t())
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                
            idx_rand = np.random.randint(0, sample_count, [self.batch_size])
            X_batch = torch.tensor(X[:, idx_rand, :]).float()
            y_hat_batch = torch.tensor(y_hat[idx_rand, :]).float()
            y_batch = self.foward(X_batch)
            loss = self.loss(y_batch, y_hat_batch)
            
            epoch_num.append(i)
            losses.append(loss.item())
        
        return (epoch_num, losses)
    
    
    def train_time_series(self, X, y_hat, max_epoch, max_epoch_per_time):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")
        
        # setting normalizer
        # flatten X and y_hat
        X_flat = np.concatenate(X, axis=0)
        y_hat_flat = np.concatenate(y_hat, axis=0)
        
        self.NMR.set_mean_and_sd(torch.tensor(X_flat), torch.tensor(y_hat_flat))
        
        
        full_losses = np.array([])

        T_running = utl.get_runtime_marker()
        for i in np.arange(max_epoch):
            for time in np.arange(9):
                (x_axis, losses) = self.train(X[:time+1, :, :], y_hat[time, :, :], max_epoch_per_time)
                full_losses = np.concatenate((full_losses, losses))
            progress_bar.update(i + 1, "epoch {0}".format(i))
            
        return (np.arange(len(full_losses)) + 1, full_losses)

#### FC

In [ ]:
class MLTCC_Model_FC_Normalized:
    def __init__(self):
        self.NMR = utl.Normalizer()
        
        self.input_size = 13
        self.output_size = 1
        self.batch_size = 10000
        self.step_size = 1e-3
        
        h1 = 20
        h2 = 40
        h3 = 20
        
        W0_rand = 1/np.sqrt(self.input_size)
        W0_init = np.random.uniform(-W0_rand, W0_rand, [h1, self.input_size])

        b0_rand = 1/np.sqrt(1)
        b0_init = np.random.uniform(-b0_rand, b0_rand, [h1, 1])

        W1_rand = 1/np.sqrt(h1)
        W1_init = np.random.uniform(-W1_rand, W1_rand, [h2, h1])

        b1_rand = 1/np.sqrt(1)
        b1_init = np.random.uniform(-b1_rand, b1_rand, [h2, 1])
        
        W2_rand = 1/np.sqrt(h2)
        W2_init = np.random.uniform(-W1_rand, W1_rand, [h3, h2])

        b2_rand = 1/np.sqrt(1)
        b2_init = np.random.uniform(-b1_rand, b1_rand, [h3, 1])
        
        W3_rand = 1/np.sqrt(h3)
        W3_init = np.random.uniform(-W1_rand, W1_rand, [self.output_size, h3])

        b3_rand = 1/np.sqrt(1)
        b3_init = np.random.uniform(-b1_rand, b1_rand, [self.output_size, 1])

        self.W0 = torch.tensor(W0_init, requires_grad=True)
        self.b0 = torch.tensor(b0_init, requires_grad=True)
        self.W1 = torch.tensor(W1_init, requires_grad=True)
        self.b1 = torch.tensor(b1_init, requires_grad=True)
        self.W2 = torch.tensor(W2_init, requires_grad=True)
        self.b2 = torch.tensor(b2_init, requires_grad=True)
        self.W3 = torch.tensor(W3_init, requires_grad=True)
        self.b3 = torch.tensor(b3_init, requires_grad=True)
#         self.fc1 = nn.Linear(self.input_size, h1)
#         self.fc2 = nn.Linear(h1, h2)
#         self.fc3 = nn.Linear(h2, self.output_size)
        
        self.optim = torch.optim.Adam(
            [
                self.W0, self.b0,
                self.W1, self.b1,
                self.W2, self.b2,
                self.W3, self.b3
            ],
#             [
#                 self.fc1.weight, self.fc1.bias,
#                 self.fc2.weight, self.fc2.bias,
#                 self.fc3.weight, self.fc3.bias,
#             ],
            lr=self.step_size
        )
        
    def foward(self, X):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
        y := [TCC],                  (sample_size, 1) like tensor
        """
        # W1 * (relu(W0 * X + b0)) + b1
        
#         X_ = X.clone().type(torch.FloatTensor)
        
#         X_ = F.relu(self.fc1(X_))
#         X_ = F.relu(self.fc2(X_))
#         X_ = self.fc3(X_)
        
#         return X_
#         X = torch.tanh(torch.matmul(self.W0, X.t()) + self.b0)

        X = self.NMR.normalize_input(X)
    
        X = torch.relu(torch.matmul(self.W0, X.t()) + self.b0)
        X = torch.relu(torch.matmul(self.W1, X) + self.b1)
        X = torch.relu(torch.matmul(self.W2, X) + self.b2)
        X = torch.matmul(self.W3, X) + self.b3
        return self.NMR.denormalize_output(X.t())
    
    def loss(self, y, y_hat):
        """
        y := [TCC],                  (sample_size, 1) like tensor
        y_hat := [TCC],              (sample_size, 1) like tensor
        """
        return F.mse_loss(y, y_hat)
    
    def train(self, X, y_hat, max_epoch):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")
        
        self.NMR.set_mean_and_sd(torch.tensor(X), torch.tensor(y_hat))
        
        sample_count = X.shape[0]
        
        epoch_num = []
        losses = []
        
        for i in np.arange(max_epoch):
            for idx_batch in np.arange(int(sample_count / self.batch_size)) * self.batch_size:
                X_batch = torch.tensor(X[idx_batch:idx_batch + self.batch_size, :])
                y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                
            idx_rand = np.random.randint(0, sample_count, [self.batch_size])
            X_batch = torch.tensor(X[idx_rand, :])
            y_hat_batch = torch.tensor(y_hat[idx_rand, :])
            y_batch = self.foward(X_batch)
            loss = self.loss(y_batch, y_hat_batch)
            
            epoch_num.append(i)
            losses.append(loss.item())
            
            progress_bar.update(i + 1, "epoch {0}".format(i))
        
        return (epoch_num, losses)

#### FC with TCC Feedback

In [ ]:
class MLTCC_Model_FC_Normalized_TCC:
    def __init__(self):
        self.NMR = utl.Normalizer()
        
        self.input_size = 14
        self.output_size = 1
        self.batch_size = 10000
        self.step_size = 1e-3
        
        h1 = 20
        h2 = 40
        h3 = 20
        
        W0_rand = 1/np.sqrt(self.input_size)
        W0_init = np.random.uniform(-W0_rand, W0_rand, [h1, self.input_size])

        b0_rand = 1/np.sqrt(1)
        b0_init = np.random.uniform(-b0_rand, b0_rand, [h1, 1])

        W1_rand = 1/np.sqrt(h1)
        W1_init = np.random.uniform(-W1_rand, W1_rand, [h2, h1])

        b1_rand = 1/np.sqrt(1)
        b1_init = np.random.uniform(-b1_rand, b1_rand, [h2, 1])
        
        W2_rand = 1/np.sqrt(h2)
        W2_init = np.random.uniform(-W1_rand, W1_rand, [h3, h2])

        b2_rand = 1/np.sqrt(1)
        b2_init = np.random.uniform(-b1_rand, b1_rand, [h3, 1])
        
        W3_rand = 1/np.sqrt(h3)
        W3_init = np.random.uniform(-W1_rand, W1_rand, [self.output_size, h3])

        b3_rand = 1/np.sqrt(1)
        b3_init = np.random.uniform(-b1_rand, b1_rand, [self.output_size, 1])

        self.W0 = torch.tensor(W0_init, requires_grad=True)
        self.b0 = torch.tensor(b0_init, requires_grad=True)
        self.W1 = torch.tensor(W1_init, requires_grad=True)
        self.b1 = torch.tensor(b1_init, requires_grad=True)
        self.W2 = torch.tensor(W2_init, requires_grad=True)
        self.b2 = torch.tensor(b2_init, requires_grad=True)
        self.W3 = torch.tensor(W3_init, requires_grad=True)
        self.b3 = torch.tensor(b3_init, requires_grad=True)
#         self.fc1 = nn.Linear(self.input_size, h1)
#         self.fc2 = nn.Linear(h1, h2)
#         self.fc3 = nn.Linear(h2, self.output_size)
        
        self.optim = torch.optim.Adam(
            [
                self.W0, self.b0,
                self.W1, self.b1,
                self.W2, self.b2,
                self.W3, self.b3
            ],
#             [
#                 self.fc1.weight, self.fc1.bias,
#                 self.fc2.weight, self.fc2.bias,
#                 self.fc3.weight, self.fc3.bias,
#             ],
            lr=self.step_size
        )
        
    def foward(self, X):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
        y := [TCC],                  (sample_size, 1) like tensor
        """
        # W1 * (relu(W0 * X + b0)) + b1
        
#         X_ = X.clone().type(torch.FloatTensor)
        
#         X_ = F.relu(self.fc1(X_))
#         X_ = F.relu(self.fc2(X_))
#         X_ = self.fc3(X_)
        
#         return X_
#         X = torch.tanh(torch.matmul(self.W0, X.t()) + self.b0)

        X = self.NMR.normalize_input(X)
    
        X = torch.relu(torch.matmul(self.W0, X.t()) + self.b0)
        X = torch.relu(torch.matmul(self.W1, X) + self.b1)
        X = torch.relu(torch.matmul(self.W2, X) + self.b2)
        X = torch.matmul(self.W3, X) + self.b3
        return self.NMR.denormalize_output(X.t())
    
    def loss(self, y, y_hat):
        """
        y := [TCC],                  (sample_size, 1) like tensor
        y_hat := [TCC],              (sample_size, 1) like tensor
        """
        return F.mse_loss(y, y_hat)
    
    def train(self, X, y_hat, max_epoch):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")
        
        self.NMR.set_mean_and_sd(torch.tensor(X), torch.tensor(y_hat))
        
        sample_count = X.shape[0]
        
        epoch_num = []
        losses = []
        
        for i in np.arange(max_epoch):
            for idx_batch in np.arange(int(sample_count / self.batch_size)) * self.batch_size:
                X_batch = torch.tensor(X[idx_batch:idx_batch + self.batch_size, :])
                y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                
            idx_rand = np.random.randint(0, sample_count, [self.batch_size])
            X_batch = torch.tensor(X[idx_rand, :])
            y_hat_batch = torch.tensor(y_hat[idx_rand, :])
            y_batch = self.foward(X_batch)
            loss = self.loss(y_batch, y_hat_batch)
            
            epoch_num.append(i)
            losses.append(loss.item())
            
            progress_bar.update(i + 1, "epoch {0}".format(i))
        
        return (epoch_num, losses)

#### Linear

In [ ]:
class MLTCC_Model_Linear_Normalized:
    def __init__(self):
        self.NMR = utl.Normalizer()
        
        self.input_size = 13
        self.output_size = 1
        self.batch_size = 50000
        self.step_size = 1e-3
        
        W0_rand = 1/np.sqrt(self.input_size)
        W0_init = np.random.uniform(-W0_rand, W0_rand, [self.output_size, self.input_size])

        b0_rand = 1/np.sqrt(1)
        b0_init = np.random.uniform(-b0_rand, b0_rand, [self.output_size, 1])

        self.W0 = torch.tensor(W0_init, requires_grad=True)
        self.b0 = torch.tensor(b0_init, requires_grad=True)
        
        self.optim = torch.optim.Adam(
            [
                self.W0, self.b0
            ],
            lr=self.step_size
        )
        
    def foward(self, X):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
        y := [TCC],                  (sample_size, 1) like tensor
        """
        
        X = self.NMR.normalize_input(X)
    
        X = torch.matmul(self.W0, X.t()) + self.b0
        
        return self.NMR.denormalize_output(X.t())
    
    def loss(self, y, y_hat):
        """
        y := [TCC],                  (sample_size, 1) like tensor
        y_hat := [TCC],              (sample_size, 1) like tensor
        """
        return F.mse_loss(y, y_hat)
    
    def train(self, X, y_hat, max_epoch):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")
        
        self.NMR.set_mean_and_sd(torch.tensor(X), torch.tensor(y_hat))
        
        sample_count = X.shape[0]
        
        epoch_num = []
        losses = []
        
        for i in np.arange(max_epoch):
            for idx_batch in np.arange(int(sample_count / self.batch_size))* self.batch_size:
                X_batch = torch.tensor(X[idx_batch:idx_batch + self.batch_size, :])
                y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                
            idx_rand = np.random.randint(0, sample_count, [self.batch_size])
            X_batch = torch.tensor(X[idx_rand, :])
            y_hat_batch = torch.tensor(y_hat[idx_rand, :])
            y_batch = self.foward(X_batch)
            loss = self.loss(y_batch, y_hat_batch)
            
            epoch_num.append(i)
            losses.append(loss.item())
            
            progress_bar.update(i + 1, "epoch {0}".format(i))
        
        return (epoch_num, losses)

#### Linear with TCC Feedback

In [ ]:
class MLTCC_Model_Linear_Normalized_TCC:
    def __init__(self):
        self.NMR = utl.Normalizer()
        
        self.input_size = 14
        self.output_size = 1
        self.batch_size = 50000
        self.step_size = 1e-3
        
        W0_rand = 1/np.sqrt(self.input_size)
        W0_init = np.random.uniform(-W0_rand, W0_rand, [self.output_size, self.input_size])

        b0_rand = 1/np.sqrt(1)
        b0_init = np.random.uniform(-b0_rand, b0_rand, [self.output_size, 1])

        self.W0 = torch.tensor(W0_init, requires_grad=True)
        self.b0 = torch.tensor(b0_init, requires_grad=True)
        
        self.optim = torch.optim.Adam(
            [
                self.W0, self.b0
            ],
            lr=self.step_size
        )
        
    def foward(self, X):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like tensor
        y := [TCC],                  (sample_size, 1) like tensor
        """
        
        X = self.NMR.normalize_input(X)
    
        X = torch.matmul(self.W0, X.t()) + self.b0
        
        return self.NMR.denormalize_output(X.t())
    
    def loss(self, y, y_hat):
        """
        y := [TCC],                  (sample_size, 1) like tensor
        y_hat := [TCC],              (sample_size, 1) like tensor
        """
        return F.mse_loss(y, y_hat)
    
    def train(self, X, y_hat, max_epoch):
        """
        X := [LTS, SST, Subsidence], (sample_size, 4) like matrix
        y_hat := [TCC],              (sample_size, 1) like matrix
        """
        progress_bar = utl.TimedProgressBar(max_epoch).update(msg="Initialization")
        
        self.NMR.set_mean_and_sd(torch.tensor(X), torch.tensor(y_hat))
        
        sample_count = X.shape[0]
        
        epoch_num = []
        losses = []
        
        for i in np.arange(max_epoch):
            for idx_batch in np.arange(int(sample_count / self.batch_size))* self.batch_size:
                X_batch = torch.tensor(X[idx_batch:idx_batch + self.batch_size, :])
                y_hat_batch = torch.tensor(y_hat[idx_batch:idx_batch + self.batch_size, :])
                y_batch = self.foward(X_batch)
                loss = self.loss(y_batch, y_hat_batch)
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                
            idx_rand = np.random.randint(0, sample_count, [self.batch_size])
            X_batch = torch.tensor(X[idx_rand, :])
            y_hat_batch = torch.tensor(y_hat[idx_rand, :])
            y_batch = self.foward(X_batch)
            loss = self.loss(y_batch, y_hat_batch)
            
            epoch_num.append(i)
            losses.append(loss.item())
            
            progress_bar.update(i + 1, "epoch {0}".format(i))
        
        return (epoch_num, losses)

***
### Process

In [ ]:
def benchmark_procedure():
    ### Data

    (idx_training_set, idx_test_set) = get_training_and_testing_set()


    ### Training

    #### GRU
    print("---------------------- GRU ----------------------")

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data(idx_test_set, idx_training_set)

    tcc_model_gru = MLTCC_Model_GRU_Normalized()

    (x_axis, losses) = tcc_model_gru.train_time_series(X_train, y_hat_train, 50, 2)

    # plot(x_axis, losses, "epoch", "losses", "loss", "epoch vs. losses")

    losses[-1]

    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_gru.foward(tsr(X_test)).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][:, 0]

    plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "GRU")

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "GRU")

    # for idx in np.arange(9):
    #     plot_y_fit = tcc_model_gru.foward(tsr(X_test[:idx+1])).detach().numpy()[:, 0]
    #     plot_y_hat = y_hat_test[idx][:, 0]
    #     plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "T{0}".format(idx))

    # y_fit_series = []
    # for idx in np.arange(9):
    #     y_fit_series.append(tcc_model_gru.foward(tsr(X_test[:idx+1])).detach().numpy())

    # y_fit_series = np.array(y_fit_series)

    # for i in np.arange(10):
    #     idx = np.random.randint(0, y_hat_test.shape[1])
    #     plot(
    #         np.arange(9), 
    #         (y_fit_series[:, idx, 0], y_hat_test[:, idx, 0]), 
    #         x_label="T", 
    #         y_label="TCC %", 
    #         legend=["fit", "true"], 
    #         title="Time Series Prediction, Sample: {0}".format(idx)
    #     )

    #### GRU with Feedback
    print("\n---------------------- GRU with Feedback ----------------------")

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data_TCC_feedback(idx_test_set, idx_training_set)

    tcc_model_gru_TCC = MLTCC_Model_GRU_Normalized_TCC()

    (x_axis, losses) = tcc_model_gru_TCC.train_time_series(X_train, y_hat_train, 50, 2)

    # plot(x_axis, losses, "epoch", "losses", "loss", "epoch vs. losses")

    losses[-1]

    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_gru_TCC.foward(tsr(X_test)).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][:, 0]

    plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "GRU with TCC Feedback Prediction")

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "GRU with TCC Feedback Prediction")

    # for idx in np.arange(9):
    #     plot_y_fit = tcc_model_gru_TCC.foward(tsr(X_test[:idx+1])).detach().numpy()[:, 0]
    #     plot_y_hat = y_hat_test[idx][:, 0]
    #     plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "T{0}".format(idx))

    # y_fit_series = []
    # for idx in np.arange(9):
    #     y_fit_series.append(tcc_model_gru_TCC.foward(tsr(X_test[:idx+1])).detach().numpy())

    # y_fit_series = np.array(y_fit_series)

    # for i in np.arange(10):
    #     idx = np.random.randint(0, y_hat_test.shape[1])
    #     plot(
    #         np.arange(9), 
    #         (y_fit_series[:, idx, 0], y_hat_test[:, idx, 0]), 
    #         x_label="T", 
    #         y_label="TCC %", 
    #         legend=["fit", "true"], 
    #         title="Time Series Prediction, Sample: {0}".format(idx)
    #     )

    #### FC
    print("\n---------------------- FC ----------------------")

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data(idx_test_set, idx_training_set)

    X_train_FC = flatten_time_axis(X_train)
    y_hat_train_FC = flatten_time_axis(y_hat_train)
    X_test_FC = flatten_time_axis(X_test)
    y_hat_test_FC = flatten_time_axis(y_hat_test)

    tcc_model_FC = MLTCC_Model_FC_Normalized()

    # (x_axis, losses) = tcc_model_FC.train(X_train_FC, y_hat_train_FC, 100)
    (x_axis, losses) = tcc_model_FC.train(X_train_FC, y_hat_train_FC, 2000)

    # plot(x_axis, losses, "epoch", "losses", "loss", "epoch vs. losses")

    losses[-1]

    tcc_model_FC.loss(tcc_model_FC.foward(tsr(X_test_FC)), tsr(y_hat_test_FC)).item()

    tcc_model_FC.loss(tsr(np.random.randint(0, 100, [y_hat_test_FC.shape[0], 1])), tsr(y_hat_test_FC)).item()

    # idx_sample = np.random.choice(np.arange(X_test_FC.shape[0]), [1000], replace=False)
    idx_sample = np.arange(X_test[-1].shape[0])  # All sample


    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_FC.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "Fully Connected Neural Network Prediction")

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Fully Connected Neural Network Prediction")

    #### FC with Feedback
    print("\n---------------------- FC with Feedback ----------------------")

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data_TCC_feedback(idx_test_set, idx_training_set)

    X_train_FC = flatten_time_axis(X_train)
    y_hat_train_FC = flatten_time_axis(y_hat_train)
    X_test_FC = flatten_time_axis(X_test)
    y_hat_test_FC = flatten_time_axis(y_hat_test)

    tcc_model_FC_TCC = MLTCC_Model_FC_Normalized_TCC()

    # (x_axis, losses) = tcc_model_FC_TCC.train(X_train_FC, y_hat_train_FC, 100)
    (x_axis, losses) = tcc_model_FC_TCC.train(X_train_FC, y_hat_train_FC, 2000)

    # plot(x_axis, losses, "epoch", "losses", "loss", "epoch vs. losses")

    losses[-1]

    tcc_model_FC.loss(tcc_model_FC_TCC.foward(tsr(X_test_FC)), tsr(y_hat_test_FC)).item()

    tcc_model_FC.loss(tsr(np.random.randint(0, 100, [y_hat_test_FC.shape[0], 1])), tsr(y_hat_test_FC)).item()

    # idx_sample = np.random.choice(np.arange(X_test_FC.shape[0]), [1000], replace=False)
    idx_sample = np.arange(X_test[-1].shape[0])  # All sample


    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_FC_TCC.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "Fully Connected Neural Network with TCC Feedback")

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Fully Connected Neural Network with TCC Feedback")

    #### Linear
    print("\n---------------------- Linear ----------------------")

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data(idx_test_set, idx_training_set)

    X_train_Linear = flatten_time_axis(X_train)
    y_hat_train_Linear = flatten_time_axis(y_hat_train)
    X_test_Linear = flatten_time_axis(X_test)
    y_hat_test_Linear = flatten_time_axis(y_hat_test)

    tcc_model_Linear = MLTCC_Model_Linear_Normalized()

    (x_axis, losses) = tcc_model_Linear.train(X_train_Linear, y_hat_train_Linear, 500)

    # plot(x_axis, losses, "epoch", "losses", "loss", "epoch vs. losses")

    losses[-1]

    tcc_model_Linear.loss(tcc_model_Linear.foward(tsr(X_test_Linear)), tsr(y_hat_test_Linear)).item()

    tcc_model_Linear.loss(tsr(np.random.randint(0, 100, [y_hat_test_Linear.shape[0], 1])), tsr(y_hat_test_Linear)).item()

    # idx_sample = np.random.choice(np.arange(X_test_FC.shape[0]), [1000], replace=False)
    idx_sample = np.arange(X_test[-1].shape[0])  # All sample


    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_Linear.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    # # idx_sample = np.random.choice(np.arange(X_test_Linear.shape[0]), [1000], replace=False)
    # idx_sample = np.arange(X_test_Linear.shape[0])  # All sample


    # # transform them from shape[n:1] into shape[n]
    # plot_y_fit = tcc_model_Linear.foward(tsr(X_test_Linear[idx_sample])).detach().numpy()[:, 0]
    # plot_y_hat = y_hat_test_Linear[idx_sample][:, 0]

    plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "Linear Prediction")

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Linear Prediction")

    # print("Weights: {0}\nOffsets: {1}".format(tcc_model_Linear.W0, tcc_model_Linear.b0))

    #### Linear with Feedback
    print("\n---------------------- Linear with Feedback ----------------------")

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data_TCC_feedback(idx_test_set, idx_training_set)

    X_train_Linear = flatten_time_axis(X_train)
    y_hat_train_Linear = flatten_time_axis(y_hat_train)
    X_test_Linear = flatten_time_axis(X_test)
    y_hat_test_Linear = flatten_time_axis(y_hat_test)

    tcc_model_Linear_TCC = MLTCC_Model_Linear_Normalized_TCC()

    (x_axis, losses) = tcc_model_Linear_TCC.train(X_train_Linear, y_hat_train_Linear, 500)

    # plot(x_axis, losses, "epoch", "losses", "loss", "epoch vs. losses")

    losses[-1]

    tcc_model_Linear_TCC.loss(tcc_model_Linear_TCC.foward(tsr(X_test_Linear)), tsr(y_hat_test_Linear)).item()

    tcc_model_Linear_TCC.loss(tsr(np.random.randint(0, 100, [y_hat_test_Linear.shape[0], 1])), tsr(y_hat_test_Linear)).item()

    # idx_sample = np.random.choice(np.arange(X_test_FC.shape[0]), [1000], replace=False)
    idx_sample = np.arange(X_test[-1].shape[0])  # All sample


    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_Linear_TCC.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    # # idx_sample = np.random.choice(np.arange(X_test_Linear.shape[0]), [1000], replace=False)
    # idx_sample = np.arange(X_test_Linear.shape[0])  # All sample


    # # transform them from shape[n:1] into shape[n]
    # plot_y_fit = tcc_model_Linear.foward(tsr(X_test_Linear[idx_sample])).detach().numpy()[:, 0]
    # plot_y_hat = y_hat_test_Linear[idx_sample][:, 0]

    plot_diff_percentage(plot_y_hat, plot_y_fit, "true", "fit", "Linear Prediction with TCC Feedback")

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Linear Prediction with TCC Feedback")

    # print("Weights: {0}\nOffsets: {1}".format(tcc_model_Linear.W0, tcc_model_Linear.b0))

    ### Report

    r_value = {}

    y_data_record = {}

    plot_y_hat = y_hat_test[-1][:, 0]
    y_data_record["y_hat"] = plot_y_hat

    #### GRU

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data(idx_test_set, idx_training_set)

    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_gru.foward(tsr(X_test)).detach().numpy()[:, 0]


    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "GRU")

    r_value["GRU"] = calc_r_value(plot_y_hat, plot_y_fit)
    y_data_record["GRU"] = plot_y_fit

    #### GRU with Feedback

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data_TCC_feedback(idx_test_set, idx_training_set)

    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_gru_TCC.foward(tsr(X_test)).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][:, 0]

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "GRU with TCC Feedback Prediction")

    r_value["GRU_FB"] = calc_r_value(plot_y_hat, plot_y_fit)
    y_data_record["GRU_FB"] = plot_y_fit

    #### FC

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data(idx_test_set, idx_training_set)

    X_train_FC = flatten_time_axis(X_train)
    y_hat_train_FC = flatten_time_axis(y_hat_train)
    X_test_FC = flatten_time_axis(X_test)
    y_hat_test_FC = flatten_time_axis(y_hat_test)

    idx_sample = np.arange(X_test[-1].shape[0])  # All sample


    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_FC.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Fully Connected Neural Network Prediction")

    r_value["FC"] = calc_r_value(plot_y_hat, plot_y_fit)
    y_data_record["FC"] = plot_y_fit

    #### FC with Feedback

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data_TCC_feedback(idx_test_set, idx_training_set)

    X_train_FC = flatten_time_axis(X_train)
    y_hat_train_FC = flatten_time_axis(y_hat_train)
    X_test_FC = flatten_time_axis(X_test)
    y_hat_test_FC = flatten_time_axis(y_hat_test)

    # idx_sample = np.random.choice(np.arange(X_test_FC.shape[0]), [1000], replace=False)
    idx_sample = np.arange(X_test[-1].shape[0])  # All sample


    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_FC_TCC.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Fully Connected Neural Network with TCC Feedback")

    r_value["FC_FB"] = calc_r_value(plot_y_hat, plot_y_fit)
    y_data_record["FC_FB"] = plot_y_fit

    #### Linear

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data(idx_test_set, idx_training_set)

    X_train_Linear = flatten_time_axis(X_train)
    y_hat_train_Linear = flatten_time_axis(y_hat_train)
    X_test_Linear = flatten_time_axis(X_test)
    y_hat_test_Linear = flatten_time_axis(y_hat_test)

    # idx_sample = np.random.choice(np.arange(X_test_FC.shape[0]), [1000], replace=False)
    idx_sample = np.arange(X_test[-1].shape[0])  # All sample


    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_Linear.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Linear Prediction")

    r_value["Linear"] = calc_r_value(plot_y_hat, plot_y_fit)
    y_data_record["Linear"] = plot_y_fit

    #### Linear with Feedback

    ((X_train, y_hat_train), (X_test, y_hat_test)) = get_data_TCC_feedback(idx_test_set, idx_training_set)

    X_train_Linear = flatten_time_axis(X_train)
    y_hat_train_Linear = flatten_time_axis(y_hat_train)
    X_test_Linear = flatten_time_axis(X_test)
    y_hat_test_Linear = flatten_time_axis(y_hat_test)

    # idx_sample = np.random.choice(np.arange(X_test_FC.shape[0]), [1000], replace=False)
    idx_sample = np.arange(X_test[-1].shape[0])  # All sample

    # transform them from shape[n:1] into shape[n]
    plot_y_fit = tcc_model_Linear_TCC.foward(tsr(X_test[-1][idx_sample])).detach().numpy()[:, 0]
    plot_y_hat = y_hat_test[-1][idx_sample][:, 0]

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "Linear Prediction with TCC Feedback")

    r_value["Linear_FB"] = calc_r_value(plot_y_hat, plot_y_fit)
    y_data_record["Linear_FB"] = plot_y_fit

    print("\n---------------------- ERA5 ----------------------")

    df = utl.read_time_series_data("full_rank_dataset_ERA5")

    idx_test_set = np.random.choice(np.arange(df[0].shape[0]), [int(0.1 * df[0].shape[0])], False)
    idx_training_set = np.delete(np.arange(df[0].shape[0]), idx_test_set)

    time_arr = np.arange(9)

    X_full = [np.c_[df[time]['ERA5'].to_numpy()] for time in time_arr]
    y_hat_full = [np.c_[df[time]['TCC'].to_numpy()] for time in time_arr]

    X_train = np.array([X_full[time][idx_training_set] for time in time_arr])
    y_hat_train = np.array([y_hat_full[time][idx_training_set] for time in time_arr])

    X_test = np.array([X_full[time][idx_test_set] for time in time_arr])
    y_hat_test = np.array([y_hat_full[time][idx_test_set] for time in time_arr])

    # transform them from shape[n:1] into shape[n]
    plot_y_fit = X_test[-1][:, 0]
    plot_y_hat = y_hat_test[-1][:, 0]

    plot_diff_percentage_bined_average(plot_y_hat, plot_y_fit, "true", "fit", "ERA5 Prediction")

    r_value["ERA5"] = calc_r_value(plot_y_hat, plot_y_fit)
    
    y_data_record["y_hat_ERA5"] = plot_y_hat
    y_data_record["ERA5"] = plot_y_fit

    #### save data
    print("Save to File: {0}".format(
        save_result("{0}{1}{2}".format(OUTPUT_FOLDER_PATH, OUTPUT_FILE_PREFIX, utl.get_current_day_time_string()), y_data_record)
    ))

In [ ]:
for i in np.arange(100):
    T = utl.get_runtime_marker()
    benchmark_procedure()
    print("EPOCH {0} FINISHED, TIME: {1}".format(i + 1, utl.format_time_s_2_hms(utl.get_runtime_in_second(T))))
    print()